#### Abstract

A new iterative algorithm is developed to solve a MINLP program for operations research. The problem consists on allocating a stock of products to the possible outlets minimizing variation with respect to the previous period, and while satisfying arbitrary constrains defined on  categories that group the variables. 

#### Introduction

The optimization problem is an abstraction of the process of choosing the best possible vector $\in R^n$ from a set. In this way it encompasses many ways of decision making, and so the reasons for its ubiquitous relevance becomes clear. The general formulation is: 

$$\begin{aligned}
& \underset{x}{\text{minimize}}
& & f_0(x) \\
& \text{subject to}
& & f_i(x) \leq b_i, \; i = 1, \ldots, m.
\end{aligned}$$

If all $f_i(x)$ fulfil linearity conditions, $f_i(αx + βy) = αf_i(x) + βf_i(y)$, then the problem corresponds to a _linear program_. A more general class of problems consist of all that comply $f_i(αx + βy) ≤ αf_i(x) + βf_i(y)$ given $α, β \in [0,1], α + β = 1$, corresponding to the domain of _convex problems_. Finally, with $x \in \mathbb{Z}$, a _Mixed Integer Convex Non-Linear Program_ can be defined.

#### Problem formulation

A stock of a $p \in \mathbb{N^i}$ products has to be allocated to $s \in \mathbb{N^j}$ outlets while satisfying arbitrary equality and inequality restrictions. These intend to modulate sales in different areas, allow compliance to commercial agreements, and respond to diverse business needs. Both $p$ and $s$ are classified in multiple hierarchical levels, e.g. : _channel, area, group, delivery route..._ for $s$; _family, product, flavor..._ for $p$. One possibility for the formulation is to organize this levels into a tensor $Q \in \mathbb{N}^{channel \; \times \; area \; \times \;...,\; family \; \times \;product...}$, or tree spanning all levels and categories, but that conveys an extra memory consumption as it assigns space that is redundant, e.g. allocating a quantity of each product for each outlet when not all products are sold at all outlets. Therefore a vector is constructed as $\textbf{q} = (s_0p_0, ..., s_ip_j)$, $\textbf{q} \in \mathbb{R}^{i \times j}$ where $p_is_j$ corresponds to the quantity of product $i$ in outlet $j$, and excluding the elements $i,j$ that are not applicable for the period distribution. Continuous relaxation of the $q$ vector is allowed for faster computation. The optimization problem is formulated as:

$$\begin{aligned}
\underset{q}{\text{minimize }}
& | ( \mathbf{q} - \mathbf{q_{0}} ) \oslash  \mathbf{q_{0}} |^{2}, 
| ( \mathbf{q} - \mathbf{q_{0}} ) \oslash  \mathbf{q_{0}} |^{\infty}
\\ \text{subject to: } 
& \textbf{R}\textbf{q} = \textbf{b} \\ 
& \textbf{M}\textbf{q} \leq \textbf{d}
\end{aligned}$$

Where each row $r$ in $R \in \{0,1\}^{i \times j}$ represents a restriction and is defined as $r_i = 1$ if the corresponding element in $\textbf{q}$ is in the subset to which the restriction applies. The symbols $\odot$ and $\oslash$ corresponds to element-wise (or Hadamard) product and division, respectively.

#### Solution 1

$$\text{Init } \textbf{q*} := q_0 $$
$$\text{Repeat until convergence:}$$

$$\textbf{e} = \textbf{b} \oslash \textbf{R} \cdot \textbf{q}$$
$$\textbf{R*} = diag( \textbf{e}) \cdot \textbf{R}$$
$$\Delta_{j} = \frac{1}{i}\sum_{\{i: R*_{ij} \neq 0\}} \textbf{R*}_{ij}$$
$$\textbf{q*} = \textbf{q} \odot \Delta$$

#### Solution 2

$$\textbf{W} = diag(\textbf{q}_0)$$
$$\textbf{s} =  \textit{Least squares}[\textbf{R}\textbf{q} = \textbf{b}]$$
$$\textbf{q*} = \textbf{q}_0 - \textbf{s}$$
$$\textbf{o} = \textit{Orth}[(R \cdot W)^T]$$
$$\textbf{v} = \textbf{W} \cdot o$$
$$\textbf{x} = \textbf{o} - \textbf{v} \cdot (v^T \cdot \textbf{W}^{-2} \cdot \textbf{o})$$
$$\textbf{q} = x + s$$

For least squares, LSQR algorithm is used (c), and the orthogonal basis $o$ is computed using singular value decomposition.

sparse matrix, lsqr, orth

In [2]:
"cia"*100

'ciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciaciacia'

In [555]:
# Ejemplo: tenemos 5 clientes con 2 productos cada uno
# Las restricciones serán: el cliente 1 no puede tener mas del 30% del total 
# el presupuestp del producto 1 es 100
# el presupuesto del producto 2 es 200

# Tenemos entonces un vector binario de largo q = clientes * productos
# Cada fila representa: (cl1) p1 p2 (cl2) p1 p2 (cl3) p11 pl2 (cl4) p1 p2 (cl5) p1 p2

In [562]:
# Restricciones
R = sparse.csc_matrix([
              [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
              [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]).astype('int8')
#objetivos
b = np.array([0.3*300, 100, 200])
#var
q = np.array([1e-10]*10)

# Ahora definimos una distribucion prior: 
# En el último mes, lo vendido fue
q0 = np.random.randint(1, 100, size=10).astype(float)
idx_ineqs = np.array([0])

In [ ]:
q_ras = solve_nras(R,q0,b,idx_ineqs, precision=1)

In [564]:
check_sol(R, q_ras, b, q0, show=False)

({'Norma q-q0': 1.213331018983049,
  'Norma Aq-b': 1.0154486093979889,
  'Max': -0.15031073347143697,
  'Min': -0.5427621323911461},
 False)

In [565]:
q_ultra, res = solve_ultra(R, q0, b, idx_ineqs)
res

Thresh 1.00 Coef 5.00 {'Norma q-q0': 0.0, 'Norma Aq-b': 120.17, 'Max': 0.0, 'Min': 0.0} 
Thresh 0.92 Coef 4.60 {'Norma q-q0': 1.07, 'Norma Aq-b': 0.0, 'Max': -0.01, 'Min': -0.82} 
Thresh 0.84 Coef 4.22 {'Norma q-q0': 1.07, 'Norma Aq-b': 0.0, 'Max': -0.01, 'Min': -0.82} 


{'Norma q-q0': 1.0706258976505163,
 'Norma Aq-b': 4.0194366942304644e-14,
 'Max': -0.01180695025980114,
 'Min': -0.8151881287936131}

#### Results

![](resultsNRAS.png) ![](resultsPROJ.png)

#### References 

Belotti, P., Kirches, C., Leyffer, S., Linderoth, J., Luedtke, J., & Mahajan, A. (2013). Mixed-integer nonlinear optimization. Acta Numerica, 22, 1-131.

Boyd, S., & Vandenberghe, L. (2004). Convex optimization. Cambridge university press.

Paige, C. C., & Saunders, M. A. (1982). LSQR: An algorithm for sparse linear equations and sparse least squares. ACM Transactions on Mathematical Software (TOMS), 8(1), 43-71.